# **Optimizing AUROC loss on imbalanced dataset**

* Author: Zhuoning Yuan

**Useful Resources**:
* Website: https://libauc.org
* Github: https://github.com/Optimization-AI/LibAUC

**Reference**:  

If you find this tutorial helpful in your work,  please acknowledge our library and cite the following paper:
```
@inproceedings{yuan2021large,
  title={Large-scale robust deep auc maximization: A new surrogate loss and empirical studies on medical image classification},
  author={Yuan, Zhuoning and Yan, Yan and Sonka, Milan and Yang, Tianbao},
  booktitle={Proceedings of the IEEE/CVF International Conference on Computer Vision},
  pages={3040--3049},
  year={2021}
}

@misc{libauc2022,
      title={LibAUC: A Deep Learning Library for X-Risk Optimization.},
      author={Zhuoning Yuan, Zi-Hao Qiu, Gang Li, Dixian Zhu, Zhishuai Guo, Quanqi Hu, Bokun Wang, Qi Qi, Yongjian Zhong, Tianbao Yang},
      year={2022}
    }
```

# **Installing LibAUC**

In [1]:
!pip install libauc.whl

ERROR: libauc.whl is not a valid wheel filename.


# **Importing LibAUC**

In [2]:
import libauc

In [3]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import resnet20 as ResNet20
from libauc.datasets import CIFAR10
from libauc.utils import ImbalancedDataGenerator
from libauc.sampler import DualSampler

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

# **Reproducibility**

In [4]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# **Image Dataset**

In [5]:
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=32, crop_size=30, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([                                                
                              transforms.ToTensor(),
                              transforms.RandomCrop((crop_size, crop_size), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((image_size, image_size)),
                              ])
       self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Resize((image_size, image_size)),
                              ])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target



# **Paramaters**

In [6]:
# paramaters
SEED = 123
BATCH_SIZE = 128
imratio = 0.1 # for demo 
lr = 0.1
gamma = 500
weight_decay = 1e-4
margin = 1.0

# **Loading datasets**

In [7]:
# dataloader 
train_data, train_targets = CIFAR10(root='./data', train=True)
test_data, test_targets  = CIFAR10(root='./data', train=False)

generator = ImbalancedDataGenerator(verbose=True, random_seed=0)
(train_images, train_labels) = generator.transform(train_data, train_targets, imratio=imratio)
(test_images, test_labels) = generator.transform(test_data, test_targets, imratio=0.5) 

trainloader = torch.utils.data.DataLoader(ImageDataset(train_images, train_labels), batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
testloader = torch.utils.data.DataLoader( ImageDataset(test_images, test_labels, mode='test'), batch_size=BATCH_SIZE, shuffle=False, num_workers=1,  pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified
#SAMPLES: [27777], POS:NEG: [2777 : 25000], POS RATIO: 0.1000
#SAMPLES: [10000], POS:NEG: [5000 : 5000], POS RATIO: 0.5000


# **Creating models & AUC Optimizer**

In [16]:
# You need to include sigmoid activation in the last layer for any customized models!
model = ResNet20(pretrained=False, last_activation=None, num_classes=1)
model = model.cuda()
from libauc.losses import AUCMLoss, CrossEntropyLoss

Loss = CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr,
                                         weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[50,75], gamma=0.1)


# **Training**

In [17]:
print ('Start Training')
print ('-'*30)
best=[0,0,0,0]

for epoch in range(100):
    
     if epoch == 50 or epoch==75:
         # decrease learning rate by 10x & update regularizer
         optimizer.update_regularizer(decay_factor=10)
   
     train_pred = []
     train_true = []
     model.train()    
     for data, targets in trainloader:
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
        #  y_pred = torch.sigmoid(y_pred)
         loss = Loss(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
        
         train_pred.append(y_pred.cpu().detach().numpy())
         train_true.append(targets.cpu().detach().numpy())

     train_true = np.concatenate(train_true)
     train_pred = np.concatenate(train_pred)
    #  print(train_true)
    #  print(train_pred)
     train_auc = roc_auc_score(train_true, train_pred) 

     model.eval()
     test_pred = []
     test_true = [] 
     for j, data in enumerate(testloader):
         test_data, test_targets = data
         test_data = test_data.cuda()
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_targets.numpy())
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     val_auc =  roc_auc_score(test_true, test_pred) 
     model.train()
     scheduler.step()
     if best==[0,0,0,0] or val_auc>best[3]:
        best = [gamma, margin, train_auc, val_auc]
        torch.save(model.state_dict(), "cifar10_resnet20_CE_im10_"+str(SEED)+".pth")

     # print results
     print("epoch: {}, train_loss: {:4f}, train_auc:{:4f}, test_auc:{:4f}".format(epoch, loss.item(), train_auc, val_auc ))          

Start Training
------------------------------
epoch: 0, train_loss: 0.322058, train_auc:0.590665, test_auc:0.641085
epoch: 1, train_loss: 0.305575, train_auc:0.653737, test_auc:0.662776
epoch: 2, train_loss: 0.279321, train_auc:0.682465, test_auc:0.701008
epoch: 3, train_loss: 0.295344, train_auc:0.708377, test_auc:0.668417
epoch: 4, train_loss: 0.305138, train_auc:0.728125, test_auc:0.750032
epoch: 5, train_loss: 0.318154, train_auc:0.748305, test_auc:0.768377
epoch: 6, train_loss: 0.386805, train_auc:0.760233, test_auc:0.741898
epoch: 7, train_loss: 0.303999, train_auc:0.777376, test_auc:0.773723
epoch: 8, train_loss: 0.181465, train_auc:0.791258, test_auc:0.786716
epoch: 9, train_loss: 0.218767, train_auc:0.803981, test_auc:0.794400
epoch: 10, train_loss: 0.349656, train_auc:0.812973, test_auc:0.796529
epoch: 11, train_loss: 0.242755, train_auc:0.825000, test_auc:0.785021
epoch: 12, train_loss: 0.211447, train_auc:0.831695, test_auc:0.815711
epoch: 13, train_loss: 0.230614, train_au